In [ ]:
#reruires "rtweet" library

library("rtweet")
#requires a token

tweets<-get_timeline(user="@MicroPasts", n=1000000000,token=token) # Gets all the @MicroPasts tweets
followers<-get_followers(@MicroPasts, n = all,  token = token) # Gets user ids of all the @MicroPasts followers
friends<-get_friends(@MicroPasts, n = all,  token = token) # Gets user ids of all the accounts which @Micropasts follows
users <- unique(c(followers[,1],friends[,1])) # Puts the ids of followers and friends in the one list

# Loops through all the users ids and gets dataframes with their tweets, then attaches them to the original dataframe:

i=1
j=length(users)+1
while(i<j){
    print(i)
    df<-get_timeline(user=users[i], n=1000000000,token=token)
    tweets<-rbind(tweets,df)
    i=i+1
}

#There were some errors, so with initial run, I skipped over some users. The code below checks which ids were skipped 
#and attempts to download them again (repeat that step if necessary)
ids<-unique(tweets$user_id)
rmids<-c()
for (user in users){
    if(is.element(user, ids)==FALSE){
        rmids<-c(rmids,user)
    }
}


#Make sure that there are no duplicate rows in the dataframe:

tweets<-unique(tweets)

#Save tweets in differently named variable

microPasts<-tweets
#Subset tweets that are not MicroPasts tweets:

microPasts <-subset(microPasts,microPasts$screen_name != "MicroPasts")

#Subset only the tweets that mention @MicroPasts

microPasts$mp<-NA

i=1
j=length(microPasts$mp)+1
while(i<j){
    print(i)
    a<-microPasts$mentions_screen_name[i] #Save the contents of the mentions_screen_name column to the variable
    a<-strsplit(a, " ") #Split the screen name from a single string into a vector
    a<-unlist(a) #Simplify vector
    if(is.element ("MicroPasts",a)==TRUE){ # Check whether the tweet mentiones MicroPasts user (whether a contains (MicroPasts))
        microPasts$mp[i]<-TRUE
    }else{
        microPasts$mp[i]<-FALSE
    }
    i=i+1  
}

microPasts<-subset(microPasts,microPasts$mp==TRUE)
microPasts$mp<-NULL #Gets rid of unnecessary column

#Get the dataframe with all the user data

users <- unique(microPasts$user_id)
user_data<-lookup_users(users)